In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

In [2]:
X,y = mnist.data, mnist.target

In [3]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
X_train, y_train = knn_clf.fit(X,y)
baseline_accuracy = knn_clf.score(X_test,y_test)
baseline_accuracy

TypeError: cannot unpack non-iterable KNeighborsClassifier object